# 🏆 모델 성능 평가 파이프라인

이 노트북은 학습 중에 생성된 체크포인트(`.ckpt`) 파일의 성능을 빠르게 평가하기 위한 전용 파이프라인입니다.

**실행 방법:**
1. **"1. 환경 설정"** 셀을 실행하여 Google Drive를 마운트하고 프로젝트 경로를 설정합니다. (최초 1회)
2. **"2. 평가 실행"** 셀에서 평가하고 싶은 **`CHECKPOINT_PATH` 변수 값만 수정**합니다.
3. **"2. 평가 실행"** 셀을 실행하면, 아래 과정이 자동으로 진행됩니다.
    - `create_evaluation_results.py`가 실행되어 `test_y` 이미지를 복원하고 `result` 폴더에 `.npy` 파일로 저장합니다.
    - `evaluate.ipynb`의 핵심 로직이 실행되어 `result` 폴더와 `label` 폴더를 비교하고 최종 PSNR/SSIM 점수를 출력합니다.


### 1. 환경 설정


In [ ]:
# @title Google Drive 마운트 및 경로 설정
import os
import sys
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 💥 사용자 설정: 자신의 Google Drive 프로젝트 경로에 맞게 수정해주세요.
PROJECT_ROOT = "/content/drive/MyDrive/Data Scientist/Project/Week5/week5"

# 작업 디렉토리 변경 및 경로 추가
os.chdir(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

# 평가에 필요한 라이브러리 설치
print("Installing required libraries...")
%pip install loguru tqdm --quiet
print("Libraries installed.")

print(f"Current working directory: {os.getcwd()}")
print("Environment setup complete.")


### 2. 평가 실행


In [ ]:
# @title 체크포인트 경로 지정 및 최종 평가 실행
import shutil
from pathlib import Path
import sys
import os

# --- ⚙️ 1. 사용자 설정: 평가할 체크포인트 경로를 여기에 입력하세요. ---
# 예시: "logs_dncnn_e2e_controlled/00000_train/checkpoints/checkpoint_best.ckpt"
CHECKPOINT_PATH = "logs_dncnn_e2e_controlled/00000_train/checkpoints/checkpoint_best.ckpt" #@param {type:"string"}
# -------------------------------------------------------------------

# --- 자동으로 경로 및 폴더 설정 ---
RESULT_FOLDER = "result"
DATASET_ROOT = "dataset"

# 이전 결과가 있다면 삭제하여 정확한 평가 환경 보장
if Path(RESULT_FOLDER).exists():
    print(f"Removing old '{RESULT_FOLDER}' directory...")
    shutil.rmtree(RESULT_FOLDER)

# --- 1단계: 결과물(.npy 파일) 생성 ---
print("\n[Phase 1/2] Creating result files from checkpoint...")
# 💥 수정: 인자 이름을 --checkpoint_path로 변경하고, 변수를 {CHECKPOINT_PATH} 형식으로 전달
!python "create_evaluation_results.py" --checkpoint_path "{CHECKPOINT_PATH}" --result_dir "{RESULT_FOLDER}"

# --- 2단계: 최종 점수 계산 ---
print("\n[Phase 2/2] Calculating final score with evaluate module...")

# 현재 작업 디렉토리 저장
original_dir = os.getcwd()
dataset_dir = os.path.join(original_dir, DATASET_ROOT)

try:
    # evaluate 모듈이 있는 dataset 폴더로 이동하여 실행
    os.chdir(dataset_dir)

    # C++ 모듈 빌드
    !python setup.py build_ext --inplace --quiet

    # 💥 수정: 캐시 문제 방지를 위해 기존 모듈 삭제 후 새로 임포트
    if '_evaluate' in sys.modules:
        del sys.modules['_evaluate']
    if 'dataset._evaluate' in sys.modules:
        del sys.modules['dataset._evaluate']
    
    import _evaluate
    
    # Evaluator는 프로젝트 루트 기준으로 경로를 인식해야 함
    evaluator = _evaluate.Evaluator(
        data_root=f"../{RESULT_FOLDER}", 
        label_root="label"
    )
    evaluator.evaluate()
except Exception as e:
    print(f"An error occurred during evaluation: {e}")
    print("Please check if the checkpoint path and file structure are correct.")
finally:
    # 평가가 끝나면 원래 프로젝트 루트로 복귀
    os.chdir(original_dir)
    print(f"\nReturned to original directory: {os.getcwd()}")

